In [1]:
import pandas as pd

# 导入后台22年数据，并按季度分成子集
df_22 = pd.read_csv('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/meta_code/2022指标触发结果.csv')
df_22.drop_duplicates(inplace=True)

df_Q1_sql = df_22[df_22.quarterly == 1]
df_Q2_sql = df_22[df_22.quarterly == 2]
df_Q3_sql = df_22[df_22.quarterly == 3]

In [2]:
# 导入程序计算数据
df_Q1_py = pd.read_csv('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/dfs/新数据22Q1_触发项目及触发等级_20230105.csv')
df_Q2_py = pd.read_csv('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/dfs/新数据22Q2_触发项目及触发等级_20230105.csv')
df_Q3_py = pd.read_csv('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/dfs/新数据22Q3_触发项目及触发等级_20230105.csv')
df_Q1_py.drop_duplicates(inplace=True)
df_Q2_py.drop_duplicates(inplace=True)
df_Q3_py.drop_duplicates(inplace=True)

In [37]:
df_code = df_Q3_py[['indicators', 'code']]  # 在这里改变程序季度输入
df_code.drop_duplicates(inplace=True)

/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_39554/3487521167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code.drop_duplicates(inplace=True)


In [38]:
df_meta = pd.merge(df_Q3_sql, df_code, on='code', how='outer')  # 在这里改变后台季度输入
df_meta.rename(columns={'finnal_level':'is_trigger'}, inplace=True)

In [39]:
data0 = df_Q3_py['indicators'][df_Q3_py.is_trigger == 0].value_counts()  #
data1 = df_Q3_py['indicators'][df_Q3_py.is_trigger == 1].value_counts()  #
data2 = df_Q3_py['indicators'][df_Q3_py.is_trigger == 2].value_counts()  #

data_meta0 = df_meta['indicators'][df_meta.is_trigger == 0].value_counts()
data_meta1 = df_meta['indicators'][df_meta.is_trigger == 1].value_counts()
data_meta2 = df_meta['indicators'][df_meta.is_trigger == 2].value_counts()

In [40]:
dict0 = {'indicators':data0.index, 'count_0':data0.values}
dict1 = {'indicators':data1.index, 'count_1':data1.values}
dict2 = {'indicators':data2.index, 'count_2':data2.values}
df0 = pd.DataFrame(dict0)
df1 = pd.DataFrame(dict1)
df2 = pd.DataFrame(dict2)

dict_meta0 = {'indicators':data_meta0.index, '后台_count_0':data_meta0.values}
dict_meta1 = {'indicators':data_meta1.index, '后台_count_1':data_meta1.values}
dict_meta2 = {'indicators':data_meta2.index, '后台_count_2':data_meta2.values}
df_meta0 = pd.DataFrame(dict_meta0)
df_meta1 = pd.DataFrame(dict_meta1)
df_meta2 = pd.DataFrame(dict_meta2)

In [41]:
df = pd.merge(df0, df1, on='indicators', how='outer')
df = pd.merge(df, df2, on='indicators', how='outer')
df_sorted = df.sort_index(axis=0, ascending=True)
df_sorted.fillna(0, inplace=True)
df_sorted['total'] = df_sorted['count_0'] + df_sorted['count_1'] + df_sorted['count_2']

df_meta = pd.merge(df_meta0, df_meta1, on='indicators', how='outer')
df_meta = pd.merge(df_meta, df_meta2, on='indicators', how='outer')
df_meta_sorted = df_meta.sort_index(axis=0, ascending=True)
df_meta_sorted.fillna(0, inplace=True)
df_meta_sorted['后台_total'] = df_meta_sorted['后台_count_0'] + df_meta_sorted['后台_count_1'] + df_meta_sorted['后台_count_2']

In [42]:
df_total = pd.merge(df_sorted, df_meta_sorted, on='indicators', how='outer')
df_total

,indicators,count_0,count_1,count_2,total,后台_count_0,后台_count_1,后台_count_2,后台_total
0,QW_OPERATINGCASHFLOW,12786,180.0,740.0,13706.0,9408.0,0.0,1252.0,10660.0
1,QW_RANGE_OPERATINGCASHFLOW,12457,855.0,394.0,13706.0,9678.0,0.0,982.0,10660.0
2,QW_REVENUE_GROWTH_DELTA,10566,112.0,134.0,10812.0,9068.0,1592.0,0.0,10660.0
3,QW_GROSSPROFITMARGIN_DELTA,10452,141.0,219.0,10812.0,9148.0,1512.0,0.0,10660.0
4,QW_RANGE_REVENUE_GROWTH_DELTA,10321,236.0,255.0,10812.0,9312.0,1348.0,0.0,10660.0
...,...,...,...,...,...,...,...,...,...
90,QW_MONEY_RATIO_DELTA,4667,525.0,214.0,5406.0,4673.0,657.0,0.0,5330.0
91,QW_RECEIVABLE_RATIO_DELTA,4663,528.0,215.0,5406.0,4633.0,697.0,0.0,5330.0
92,QW_STOCK_RATIO_DELTA,4655,522.0,229.0,5406.0,4600.0,730.0,0.0,5330.0
93,QW_ROE_DELTA,4654,300.0,452.0,5406.0,4579.0,751.0,0.0,5330.0


In [43]:
df_total.to_excel('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/meta_output/22Q3_触发统计信息_230106.xlsx')

---

In [12]:
# -*- coding: utf-8 -*-
import pandas as pd
import os

# 比对阈值差别
os.chdir('..')
print(os.getcwd())
path_dfs = r"dfs/"  # 在此更改数据输入文件夹
path_meta = r"meta_code/"  # 在此更改meta_code输入文件夹

# 导入两份需要比对的数据

cmpdata = df_Q3_py
sqldata = df_Q3_sql

/Users


In [13]:
sqldata.rename(columns={'quarterly':'quarter',
                        'finnal_level':'is_trigger'}, inplace=True)

/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_40070/2236816620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sqldata.rename(columns={'quarterly':'quarter',


In [14]:
# cmpdata=cmpdata.drop(labels=['Unnamed: 0'], axis=1)
cmpdata = cmpdata.drop_duplicates()
# 将两份数据匹配
result = pd.merge(cmpdata, sqldata, how='inner', on=["entity_name", "year", "quarter", "code"])

result = result[['entity_name', 'stock_code', 'industry', 'year', 'quarter', 'indicators', 'code', 'finnal_value', 'general_trigger', 'high_trigger', 'is_trigger_origin', '指标异常值处理', '指标异常值处理结果', 'is_trigger_x', 'is_trigger_y']]
result.rename({"is_trigger_x": "is_trigger_产品", "is_trigger_y": "is_trigger_开发"}, axis=1, inplace=True)

result['is_diff'] = result.apply(lambda x: 1 if x['is_trigger_产品'] != x['is_trigger_开发'] else 0, axis=1)

result.to_excel('/Users/zhuangyuhao/PycharmProjects/SeasonRepMod/整合季报代码模型/meta_output/22Q3_前后台触发对比结果_230109.xlsx', index=False)